In [1]:
from pyrover_domain.librovers import rovers
from tqdm import tqdm
from pyrover_domain.custom_env import createEnv

import os 
from pathlib import Path
import yaml

import numpy as np

import logging

# Create and configure logger
logging.basicConfig(format='%(asctime)s %(message)s')
 
# Creating an object
logger = logging.getLogger()
 
# Setting the threshold of logger to DEBUG
logger.setLevel(logging.INFO)

print ("Setting up bindings...")

config_dir = Path(os.path.expanduser("/home/magraz/rovers/pyrover_domain/config/experiment_1_decay.yaml"))

with open(str(config_dir), 'r') as file:
    config = yaml.safe_load(file)

# 4 sectors times 2 distance types from pois and rovers
state_len = 8

env = createEnv(config)

theta = np.linspace(-0.75*np.pi, 0.25*np.pi, config["ccea"]["num_steps"])

Setting up bindings...


In [2]:
states, rewards = env.reset()

for s in (range(config["ccea"]["num_steps"])):

    for i, rover in enumerate(env.rovers()):
        print(f"ROVER: {i},  POS: {rover.position()}")

    

    for i, poi in enumerate(env.pois()):
        pack = rovers.EntityPack(
            entity = poi,
            agents = env.rovers(),
            entities = env.pois()
        )

        print(f"{s} POI: {i},  POS: {poi.position()},  VIS: {poi.visible}, SATISFIED: {poi.constraint_satisfied(pack)} VAL: {poi.value()}\n")

    radius = 21

    h = 25
    k = 25
    
    dx = -(rover.position().x - (radius * np.cos(theta[s]) + h))
    dy = -(rover.position().y - (radius * np.sin(theta[s]) + k))

    norm = np.sqrt(dx**2 + dy**2)

    norm_dx = dx/norm
    norm_dy = dy/norm

    if s == 0:
        dummy_action = [rovers.tensor([dx,dy])]
    else:
        # dummy_action = [rovers.tensor([norm_dx,norm_dy])]
        dummy_action = [rovers.tensor([0,0])]
    
    for state in states:
        obs_tensor = state.data() 
        obs_tensor.reshape((8,)) #State space is 8 dimensional
        obs_tensor = np.frombuffer(obs_tensor, dtype=np.double, count=8)

        print(f"STATE: {obs_tensor}")
    
    print(f"REW: {rewards}")

    states, rewards = env.step(dummy_action)

ROVER: 0,  POS: [25,25]

0 POI: 0,  POS: [10,10]
,  VIS: True, SATISFIED: False VAL: 80.0

0 POI: 1,  POS: [40,10]
,  VIS: True, SATISFIED: False VAL: 40.0

0 POI: 2,  POS: [40,40]
,  VIS: True, SATISFIED: False VAL: 20.0

STATE: [-1.         -1.         -1.         -1.          0.04444444 -1.
  0.17777778  0.08888889]
REW: { 0.0000000 }
ROVER: 0,  POS: [10.1508,10.1508]

1 POI: 0,  POS: [10,10]
,  VIS: True, SATISFIED: True VAL: 80.0

1 POI: 1,  POS: [40,10]
,  VIS: True, SATISFIED: False VAL: 40.0

1 POI: 2,  POS: [40,40]
,  VIS: True, SATISFIED: False VAL: 20.0

STATE: [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  1.12236309e-02 -1.00000000e+00  1.75995511e+03  4.48933782e-02]
REW: { 80.000000 }
ROVER: 0,  POS: [10.1508,10.1508]

2 POI: 0,  POS: [10,10]
,  VIS: True, SATISFIED: True VAL: 80.0

2 POI: 1,  POS: [40,10]
,  VIS: True, SATISFIED: False VAL: 40.0

2 POI: 2,  POS: [40,40]
,  VIS: True, SATISFIED: False VAL: 20.0

STATE: [-1.00000000e+00 -1.00000000e+00